In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import pyathena
from datetime import datetime
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import boto3
import s3fs
import sys
#https://wiki.bugwood.org/Botrytis_cinerea
#https://www.horticulture.com.au/globalassets/hort-innovation/resource-assets/ny15002-botrytis-fact-sheet.pdf
# rh 93-100, temp 18-23, reduces below 15 and above 25

#TODO

1. Longest pos sum -hum

2. Longest neg sum temp-dew diff

3. Longest sum ,  temp in band, score 1 if within

4. Regression class from outside forecast to GH weather


In [ ]:
#!pip install pyathena

In [ ]:
!pwd

In [ ]:
sys.path.insert(0, '../yield_prediction/fr-yield-prediction/notebooks/notebook_utils')
sys.path.insert(0, '../yield_prediction/fr-yield-prediction/src')
sys.path.insert(0, '../yield_prediction/fr-yield-prediction/src/utils')

In [ ]:
%load_ext autoreload
%autoreload 1
%aimport notebook_helpers
%aimport data_imputers
%aimport data_processors
%aimport model_data_helpers

In [ ]:
!pwd

### Config

In [ ]:
measures = ['inside_light_hr','inside_temp_hr','inside_rh_hr', 'dewpoint_hr']

ns_drop = ['datetime','date', 'time', 'hitemp', 'lowtemp', 'dewpt', 'windspeed', 'winddir', 
        'windrun', 'hispeed',  'hidir', 'windchill', 'heatindex', 'thwindex', 'thswindex', 'bar', \
       'rain', 'rainrate', 'solarenergy', 'hi_solarrad',  'uv_index', 'uv_dose', 'hi_uv', \
        'heatdd', 'cooldd', 'in_temp', 'inhum', 'in_heat', 'et', 'windsamp',  \
        'windtx', 'iss_recept', 'arcint', 'in_emc', 'in_air_density', 'source', 'ingestion_date', 'year']

ns_event_names = {'solarrad': 'inside_light_hr', 
                'tempout': 'inside_temp_hr',
                'outhum':  'inside_rh_hr',
                'in_dew': 'dewpoint_hr'
                 }

light_hours = {
    'summer': [6,19],
    'winter': [8,17]
}

In [ ]:
disease_temp_dict = {
     'botrytis': {
                  'hours' : 24,                
                  'inside_rh_hr': {"bound" : 'high',
                          "threshold_value" : 93,
                          "step_size": 2.5},         
                    
                  'inside_temp_hr': {"bound" : 'between',
                      "low_threshold_value" : 15,
                     "high_threshold_value" : 25} 
         
                },
    
    'powdery_mildew': {
                 'hours': 8,
                  'inside_rh_hr': {"bound" : 'high',
                          "threshold_value" : 90,
                            "step_size": 5}, 
                  'inside_temp_hr': {"bound" : 'low',
                          "threshold_value" : 25,
                          "step_size": 2.5}
                  },

    'late_blight': {
                 'hours': 8,
                  'inside_rh_hr': {"bound" : 'high',
                          "threshold_value" : 90,
                            "step_size": 5}, 
                  'inside_temp_hr': {"bound" : 'low',
                          "threshold_value" : 25,
                          "step_size": 2.5}
                }
}



disease_dew_dict = {
     'botrytis': {
                  'hours' : 24,                
                  'inside_rh_hr': {"bound" : 'high',
                          "threshold_value" : 93,
                          "step_size": 2.5},
         
                     'temp_dew_diff': {"bound" : 'low',
                          "threshold_value" : 0,
                          "step_size": 1.5},

#                   'inside_temp_hr': {"bound" : 'low',
#                           "threshold_value" : 25,
#                           "step_size": 2.5}
         
                  'inside_temp_hr': {"bound" : 'between',
                      "low_threshold_value" : 15,
                     "high_threshold_value" : 25} 
         
                },
    
    'powdery_mildew': {
                 'hours': 8,
                  'inside_rh_hr': {"bound" : 'high',
                          "threshold_value" : 90,
                            "step_size": 5}, 
                  'inside_temp_hr': {"bound" : 'low',
                          "threshold_value" : 25,
                          "step_size": 2.5}
                  },

    'late_blight': {
                 'hours': 8,
                  'inside_rh_hr': {"bound" : 'high',
                          "threshold_value" : 90,
                            "step_size": 5}, 
                  'inside_temp_hr': {"bound" : 'low',
                          "threshold_value" : 25,
                          "step_size": 2.5}
                }
}

In [ ]:
#PRODADD
def impute_night_light(df, light_hours, light_base_value=5):
    '''
    df = hourly data
    Light at night is around zero but this value is noisy and not ideal for data stsrndadisation.
    We set it to a base value to stabilise variance around zero.
    We flag hours during the day with near zero light for verification
    '''
    # set noisy out hours to baseline
    summer = light_hours['summer']
    winter = light_hours['winter']
    
    # initialise flag for noisy light data hours
    df['check_light_data'] = [0]*len(df)
    
    for idx, row in df.iterrows() : 
        # absolute night hours
        if (row['hour'] < summer[0]) or (row['hour'] > summer[1]):
            df.at[idx, 'inside_light_hr'] = light_base_value 
        # border line night hours     
        if (row['hour'] <= winter[0]) or (row['hour'] >= winter[1]): 
            if row['inside_light_hr'] < light_base_value:
                df.at[idx, 'inside_light_hr'] = light_base_value 
        # possibly noise data during day
        if winter[0] < row['hour'] < winter[1]:    
            if row['inside_light_hr'] < light_base_value:        
                df.at[idx, 'check_light_data'] = 1
    
    # remove noisy data during day
    # logg this 
    suspicious_pc = round(100* df['check_light_data'].mean(),3)
    first_date = df.loc[df['check_light_data'] == 1,'event_date_time'].min()
    print(f"Check {suspicious_pc}% supiscious light reading daytime from {first_date}")
    
    return df[df['check_light_data'] == 0], df[df['check_light_data'] == 1]



#PRODADD
def year_week_select_complete_days(df, hours_threshold=20):
    '''
    df = dataframe after datetime slicing
    Slicing days by datetime return midnight of last day. 
    this means only 0000Hr data for that day. these days needs to be dropped 
    from the analysis.
    Apply this method after date time slicing. This is important to in calculating seasona indexes (SI).
    Ideally 24 hours are expected in a day. Thresshold set to 20 to accomodate days with few missing hours.
    Over a number of days averaging with few missing hours randomly should not impact SI calculation
    significantly
    
    '''
    
    # for each week drop the lone MN hour days from grouping
    df['filter_index'] = df['year_week'] + df['day'].astype('str')
    days_to_keep = df.groupby('filter_index')['hour'] \
                         .count()[df.groupby('filter_index')['hour'].count() >= hours_threshold] \
                         .index


    df_out = df[df['filter_index'].isin(days_to_keep)]

    return df_out.drop(columns='filter_index')


def drop_zero_read_hours(df, measures):
    # removes hours with zero readings, possible device error
    return  df[~(df[measures] == 0).any(axis=1)]


def longest_signed_run(x_array):
    # sum of longest run of positives or negatives in array
        
    res_pos = 0 
    cnt_pos = 0
    for num in x_array:
        if num > 0:
            cnt_pos += 1
            #print(cnt_pos)
        else:
            res_pos = max(res_pos, cnt_pos)
            cnt_pos = 0
    res_pos = max(res_pos, cnt_pos)
    #print(res_pos)
    
    res_neg = 0 
    cnt_neg = 0
    for num in x_array:
        if num < 0:
            cnt_neg += 1
            #print(cnt_neg)
        else:
            res_neg = min(res_neg, cnt_neg)
            cnt_neg = 0
    res_neg = min(res_neg, cnt_neg)
    #print(res_neg)
    
    return res_pos, res_neg



def longest_signed_sum(x_array):
    # returns the sum of longest run of positives or negatives in array
        
    res_pos = 0 
    sum_pos = 0
    for num in x_array:
        if num > 0:
            sum_pos += num
            #print(sum_pos)
        else:
            res_pos = max(res_pos, sum_pos)
            sum_pos = 0
    res_pos = max(res_pos, sum_pos)
    #print(res_pos)
    
    res_neg = 0 
    sum_neg = 0
    for num in x_array:
        if num < 0:
            sum_neg += num
            #print(sum_neg)
        else:
            res_neg = min(res_neg, sum_neg)
            sum_neg = 0
    res_neg = min(res_neg, sum_neg)
    #print(res_neg)
    
    return res_pos, res_neg



def variance_score(xi, bound, threshold_value, step_size):
    score = 0
    var_ = xi - threshold_value
    if bound == 'low':
        if var_ < 0:
            score = abs(round(var_ / step_size,0))
        else:
            score = 0
    if bound == 'high':
        if var_ > 0:
            score = round(var_ / step_size,0)
        else:
            print('fuck you')
            score = 0
            print(score)
    
    return score


def variance_event(xi, bound, threshold_value, step_size):
    count = 0
    var_ = xi - threshold_value
    if bound == 'low':
        if var_ < 0:
            count = 1
        else:
            count = 0
    if bound == 'high':
        if var_ > 0:
            count = 1
        else:
            #print('fuck you')
            count = 0
 
    return count


def sel_data(data, feat, len_=10):    
    return data[feat][-len_:]   


def overall_score(scores, events, conditions=2):
    ov_score = 0
    ov_events = 0
    
    if conditions == 2:    
        for x, y in zip(scores[0],scores[1]):
            if x > 0 and y > 0:
                ov_events += 1
                tot = x+y
                ov_score += tot 
                
    
    if conditions == 3:    
        for x, y, z in zip(scores[0],scores[1], scores[2]):
            if x > 0 and y > 0 and z > 0:
                ov_events += 1
                tot = x + y + z
                ov_score += tot  
                
    return ov_score, ov_events


def model_risk_scores(data, disease, conditions=2, disease_dict=disease_dict): 
    events = []
    scores = []
    cont_run_stats = {}
    cont_sum_stats = {}    
    for feat in list(disease_dict[disease].keys())[1:]:
        dur = disease_dict[disease]['hours']
        bound = disease_dict[disease][feat]['bound']
        x = sel_data(data, feat=feat, len_=dur)
        print(x)
        score_x = []
        event_x = []
        
        if bound != 'between':
            threshold_value = disease_dict[disease][feat]['threshold_value'] 
            step_size = disease_dict[disease][feat]['step_size'] 
            x = sel_data(data, feat=feat, len_=dur)
       
            for xi in x:
                score_x.append(variance_score(xi, bound, threshold_value, step_size))
                event_x.append(variance_event(xi, bound, threshold_value, step_size))   
            cont_run , _  = longest_signed_run(event_x)        
            cont_sum , _  = longest_signed_sum(score_x)
            print("event_x :", event_x)     
            print('score_x :', score_x)
            
        elif bound == 'between':           
            # low and high bounds
            low_ = disease_dict[disease][feat]['low_threshold_value']
            high_ = disease_dict[disease][feat]['high_threshold_value'] 
           
            for xi in x:    
                score_x.append(select_constraint(xi, [low_, high_]))
            print("constraint :", score_x)

            
        cont_run_stats[feat] = cont_run
        cont_sum_stats[feat] = cont_sum
        events.append(event_x)
        scores.append(score_x)
        scores.append(cont_sum_stats)
    print('scores')
    print(scores)
      
    # all conditions met, i.e. temp low and hum high in same hour
    ov_risk_score, ov_risk_hrs = overall_score(scores, events, conditions)
    
    return ov_risk_score, ov_risk_hrs

    
def hist_hourly_miss_percent(df, history_horizon=56): 
    """
    Data can missing in anumber of ways.
    1. No data may received from the Folium for some hours or days
    2. From this data, some entries can be missing.
    The expected data hours are calculated for any given number of weeks.
    The missing percent data is calculated as number of 
    """
    expected_rows = history_horizon*24 # hours in the weeks to analyse, e.g 8
    print("Expected data hours :", expected_rows)
    miss_n = (df.isna().sum(axis=1) > 0).sum() # missing rows in received data
    print("Missing data hours in the retrieved dataset :", miss_n)
    available_rows = len(df) - miss_n # rows available for analysis
    print("Complete data hours :", available_rows)
    miss_prop = round(((expected_rows - available_rows)/ expected_rows),4)
    miss_pc = 100* miss_prop
    print(f"{miss_pc} % data hours missing")
    
    return miss_prop

#PRODADD
def select_complete_days(df, hours_threshold=20):
    '''
    df = dataframe after datetime slicing, wide for folium features
    Slicing days by datetime return midnight of last day. 
    this means only 0000Hr data for that day. these dayas needs to be dropped 
    from the analysis.
    Apply this method after date time slicing. This is important to in calculating seasona indexes (SI).
    Ideally 24 hours are expected in a day. Thresshold set to 20 to accomodate days with few missing hours.
    Over a number of days averaging with few missing hours randomly should not impact SI calculation
    significantly
    '''
    days_to_keep = df.groupby('day')['hour'] \
                     .count()[df.groupby('day')['hour'].count() >= hours_threshold] \
                     .index.to_list()
    
    return df[df['day'].isin(days_to_keep)]

def fix_hourly_data(df):
    '''
    df is dat is data with only features and not yield
    'event_date_time' column for time stamp is expected
    '''
    df = df[~pd.isna(df['event_date_time'])]
    df['event_date_time'] = pd.to_datetime(df['event_date_time'])
    return df
#OK

def sort_hourly_data(df):
    df['event_date_time'] = pd.to_datetime(df['event_date_time'])    
    return df.set_index('event_date_time').sort_index()
#OK

#PRODADD
#TODOM make sure start date, beginning of harvest week
def create_history_data(df, start_date, horizon=56, hours_threshold=20):  
    '''
    df : df
        df with datetime as index
    start_date: str in form, '2022-12-26' 
    horizon: int
            days to scan back for env patterns    
    '''
    q_start_date = pd.Timestamp(start_date) - DateOffset(days=horizon)
    q_end_date = pd.Timestamp(start_date)
    out_ = df.loc[q_start_date : q_end_date]
    
    # drop lonely midnight hour on the last day selected 
    out = select_complete_days(out_, hours_threshold)
    
    return out
# OK


def select_constraint(xi, bounds):
    if bounds[0] <= xi <= bounds[1]:
        score = 1
    else:
        score = 0
    
    return score


def weekly_incidence_data(df, disease='Botrytis'):
    return df.loc[df['disease']==disease,:]


def gh_weekly_incidence_data(df, gh, disease='botrytis'):
    return df.loc[(df['GH']==gh) &(df['disease']==disease),:]


def plot_roc_curve(true_y, y_prob):
    """
    plots the roc curve based of the probabilities
    """

    fpr, tpr, thresholds = roc_curve(true_y, y_prob)
    plt.plot(fpr, tpr)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')

### Data

In [ ]:
aws_session = boto3.session.Session(profile_name='research-sso')
s3_client = aws_session.client('s3')

In [ ]:
### NS Environment and Disease data
env_q = """select * from flattened_raw_db.naturesweet_external_environment"""

pnd_q = """ select * from flattened_raw_db.naturesweet_pests_diseases"""

mock_data = pd.read_csv("hourly_env_data.csv")
ns_env_pre = notebook_helpers.load_athena_data(query=env_q, env='research-sso')
ns_disease = notebook_helpers.load_athena_data(query=pnd_q, env='research-sso')

#### 1. Environment data

In [ ]:
ns_env_pre.columns

In [ ]:
ns_env_pre['event_date_time'] = pd.to_datetime(ns_env_pre['datetime']).dt.round('H')
ns_env_pre = ns_env_pre.rename(columns = ns_event_names)
ns_env_pre = ns_env_pre.drop(columns=ns_drop)

# create timstamp index columns
ns_env_pre['day'] = ns_env_pre['event_date_time'].dt.day_of_year
ns_env_pre['hour'] = ns_env_pre['event_date_time'].dt.hour


# impute data
ns_env, ns_check_data = impute_night_light(ns_env_pre, light_hours, light_base_value=5)
ns_env = drop_zero_read_hours(ns_env, measures)

# average by hour
# cleanup 12MN lone hour
ns_env_data = pd.DataFrame(ns_env.groupby(['year_week','event_date_time','day','hour']).mean()) \
                .reset_index()
#ns_env_data = year_week_select_complete_days(ns_env_data, hours_threshold=20)

ns_env_data['temp_dew_diff'] = ns_env_data['dewpoint_hr'] - ns_env_data['inside_temp_hr']

#### 2.  Disease data

In [ ]:
pd2018 = pd.read_excel('pestdiseases.xlsx', sheet_name='PYE 2018')
pd2019 = pd.read_excel('pestdiseases.xlsx', sheet_name='PYE 2019')
pd2020 = pd.read_excel('pestdiseases.xlsx', sheet_name='PYE 2020')

In [ ]:
pd2018['GH'] = pd2018['GH'].fillna(method="ffill")
pd2019['GH'] = pd2019['GH'].fillna(method="ffill")
pd2020['GH'] = pd2020['GH'].fillna(method="ffill")

In [ ]:
pd2018 = pd2018[pd2018['disease']!='year_week']
pd2019 = pd2018[pd2019['disease']!='year_week']
pd2020 = pd2018[pd2020['disease']!='year_week']

pd2018 = pd2018.drop_duplicates()
pd2019 = pd2019.drop_duplicates()
pd2020 = pd2020.drop_duplicates()

In [ ]:
pd2018

In [ ]:
pd2018.drop_duplicates()

In [ ]:
pd2018_inc = pd.DataFrame(pd.wide_to_long(pd2018, stubnames='W',i=['GH','disease'], j='week')).reset_index() # sep='_', suffix=r'\w+')
pd2018_inc['year_week'] = ['2018-' + '0' + str(x) if x < 10 else '2018-' + str(x) for x in pd2018_inc['week']]
pd2018_inc.rename(columns={"W": 'incidence'}, inplace=True)

pd2019_inc = pd.DataFrame(pd.wide_to_long(pd2019, stubnames='W',i=['GH','disease'], j='week')).reset_index() # sep='_', suffix=r'\w+')
pd2019_inc['year_week'] = ['2019-' + '0' + str(x) if x < 10 else '2019-' + str(x) for x in pd2019_inc['week']]
pd2019_inc.rename(columns={"W": 'incidence'}, inplace=True)

pd2020_inc = pd.DataFrame(pd.wide_to_long(pd2020, stubnames='W',i=['GH','disease'], j='week')).reset_index() # sep='_', suffix=r'\w+')
pd2020_inc['year_week'] = ['2020-' + '0' + str(x) if x < 10 else '2020-' + str(x) for x in pd2020_inc['week']]
pd2020_inc.rename(columns={"W": 'incidence'}, inplace=True)

In [ ]:
pd2018_inc

In [ ]:
all_disease = pd.concat([pd2018_inc, pd2019_inc, pd2020_inc])

In [ ]:
all_disease

In [ ]:
all_disease.disease.unique() # 'cenicilla', 'fulvia

#### 3.  combine disease and env data

In [ ]:
env_disease = ns_env_data.merge(all_disease, on='year_week', suffixes=["","_"])
env_disease

In [ ]:
# test data pull
test_gh = gh_weekly_incidence_data(env_disease, gh='J09', disease='Botrytis')
test_gh[:300]

In [ ]:
test_gh.incidence.plot()

### Botrytis Risk

In [ ]:
botrytis = weekly_incidence_data(env_disease, disease='Botrytis')
botrytis[:300]

In [ ]:
botrytis.dewpoint_hr

In [ ]:
#test_all.incidence.min(), test_all.incidence.max()

In [ ]:
botrytis.incidence.plot()

In [ ]:
botrytis['infected'] = [1 if x>0 else 0 for x in botrytis['incidence'] ]

In [ ]:
# seperate weeks with Positive and Negative Botrytis

In [ ]:
bot_pos = botrytis[botrytis['infected']==1]
bot_neg = botrytis[botrytis['infected']==0]

In [ ]:
bot_neg

In [ ]:
bot_pos['date_str'] = bot_pos['event_date_time'].dt.date.astype(str)
bot_pos['gh_date_str'] = bot_pos['GH'] +  "_"  +  bot_pos['year_week']  +  "_"  +  bot_pos['date_str']

bot_neg['date_str'] = bot_neg['event_date_time'].dt.date.astype(str)
bot_neg['gh_date_str'] = bot_neg['GH']  +  "_"  +  bot_neg['year_week']  +  "_"  +  bot_neg['date_str']

In [ ]:
bot_pos.groupby(['GH', 'year_week'])['gh_date_str'].min()

In [ ]:
pos_dates = bot_pos['gh_date_str'].unique()

pos_week = []
pos_event_date = []
pos_scores = []
pos_hours = []
for date in pos_dates:
    dat_ = bot_pos[bot_pos['gh_date_str']==date]
    if len(dat_)==24:
        pos_week.append(dat_['year_week'].values[0])
        pos_event_date.append(date)
        risk_score, ev_hours = model_risk_scores(dat_, 'botrytis', disease_dict=disease_dict)
        pos_scores.append(risk_score) 
        pos_hours.append(ev_hours)
        print(len(pos_dates), len(pos_scores), len(pos_hours))
        
pos_res = pd.DataFrame(
            {'pos_week': pos_week,
             'pos_dates': pos_event_date, 
             'pos_scores': pos_scores,
             'pos_hours': pos_hours
             })
        

In [ ]:
pos_res.to_csv("pos_results.csv")
pos_res

In [ ]:
neg_dates = bot_neg['gh_date_str'].unique()
#neg_dates = ['B01_2019-38_2019-09-19']

neg_week = []
neg_event_date = []
neg_scores = []
neg_hours = []
for date in neg_dates:
    dat_ = bot_neg[bot_neg['gh_date_str']==date]
    if len(dat_)==24:
        neg_week.append(dat_['year_week'].values[0])
        neg_event_date.append(date)
        risk_score, ev_hours = model_risk_scores(dat_, 'botrytis', disease_dict=disease_dict)
        neg_scores.append(risk_score) 
        neg_hours.append(ev_hours)
        
        
neg_res = pd.DataFrame(
            {'neg_week': neg_week,
             'neg_dates': neg_event_date, 
             'neg_scores': neg_scores,
             'neg_hours': neg_hours
             })  

In [ ]:
neg_res.to_csv("neg_results.csv")
neg_res

### Model Evaluation

#### 1. Any days

In [ ]:
pos_index = [1]*len(pos_scores)
pos_roc = pd.DataFrame({'score': pos_scores, 'infected': pos_index })
neg_index = [0]*len(neg_scores)
neg_roc = pd.DataFrame({'score': neg_scores, 'infected': neg_index })
bot_poc = pd.concat([pos_roc, neg_roc])

In [ ]:
plot_roc_curve(bot_poc['infected'], bot_poc['score'])

In [ ]:
roc_auc_score(bot_poc['infected'], bot_poc['score'])

In [ ]:
bot_poc

#### 2. Max infection day

In [ ]:
neg_max = neg_res.groupby('neg_week')['neg_scores'].max()
pos_max = pos_res.groupby('pos_week')['pos_scores'].max()

In [ ]:
# filteres
pos_index = [1]*len(pos_max)
pos_roc_max = pd.DataFrame({'score': pos_max, 'infected': pos_index })
neg_index = [0]*len(neg_max)
neg_roc_max = pd.DataFrame({'score': neg_max, 'infected': neg_index })
bot_poc_max = pd.concat([pos_roc_max, neg_roc_max])

In [ ]:
plot_roc_curve(bot_poc_max['infected'], bot_poc_max['score'])

In [ ]:
roc_auc_score(bot_poc_max['infected'], bot_poc_max['score'])

In [ ]:
pos_max.plot()

In [ ]:
neg_max.plot()

## ------------- PLAY ---------------------

In [ ]:
# THE END
print(datetime.now())

In [ ]:
pos_dates = bot_pos['gh_date_str'].unique()

pos_week = []
pos_event_date = []
pos_scores = []
pos_hours = []
for date in pos_dates[-10:]:
    dat_ = bot_pos[bot_pos['gh_date_str']==date]
    if len(dat_)==24:
        pos_week.append(dat_['year_week'].values[0])
        pos_event_date.append(date)
        risk_score, ev_hours, cont_score = model_risk_scores(dat_, 'botrytis', disease_dict=disease_dict)
  

In [ ]:
 risk_score

In [ ]:
dat_

In [ ]:
def model_risk_scores(data, disease, conditions=2, disease_dict=disease_temp_dict): 
    events = []
    scores = []
    cont_scores = []
    cont_run_stats = {}
    cont_sum_stats = {}    
    for feat in list(disease_dict[disease].keys())[1:]:
        dur = disease_dict[disease]['hours']
        bound = disease_dict[disease][feat]['bound']
        x = sel_data(data, feat=feat, len_=dur)
        print(x)
        score_x = []
        event_x = []
        
        if bound != 'between':
            threshold_value = disease_dict[disease][feat]['threshold_value'] 
            step_size = disease_dict[disease][feat]['step_size'] 
            x = sel_data(data, feat=feat, len_=dur)
       
            for xi in x:
                score_x.append(variance_score(xi, bound, threshold_value, step_size))
                event_x.append(variance_event(xi, bound, threshold_value, step_size))   
          
            print("event_x :", event_x)     
            print('score_x :', score_x)
            
        elif bound == 'between':           
            # low and high bounds
            low_ = disease_dict[disease][feat]['low_threshold_value']
            high_ = disease_dict[disease][feat]['high_threshold_value'] 
           
            for xi in x:    
                score_x.append(select_constraint(xi, [low_, high_]))
            print("constraint :", score_x)
       
        events.append(event_x)
        scores.append(score_x)     
        
    print('scores')
    print(scores)
      
    # all conditions met, i.e. temp low and hum high in same hour
    ov_risk_score, ov_risk_hrs = overall_score_continous(scores, events, conditions)
    
    return ov_risk_score, ov_risk_hrs


def overall_score_sparse(scores, events, conditions=2):
    ov_score = 0
    ov_events = 0
    
    if conditions == 2:    
        for x, y in zip(scores[0],scores[1]):
            if x > 0 and y > 0:
                ov_events += 1
                tot = x+y
                ov_score += tot 
                
    
    if conditions == 3:    
        for x, y, z in zip(scores[0],scores[1], scores[2]):
            if x > 0 and y > 0 and z > 0:
                ov_events += 1
                tot = x + y + z
                ov_score += tot  
                
    return ov_score, ov_events


def overall_score_continous(scores, events, conditions=2):
    ov_score = 0
    ov_events = 0
    
    if conditions == 2: 
        tot = []
        for x, y in zip(scores[0],scores[1]):
            if x > 0 and y > 0:                
                ov_events += 1               
                tot.append(x+y)
            else:
                tot.append(0)
        print(tot)        
        ov_score, _ =  longest_signed_sum(tot)
        print(ov_score)

    
    if conditions == 3:    
        for x, y, z in zip(scores[0],scores[1], scores[2]):
            tot = []
            if x > 0 and y > 0 and z > 0:
                ov_events += 1
                tot.append(x + y + z)
        ov_score, _ =  longest_signed_sum(tot)
        print(ov_score)

    return ov_score, ov_events

In [ ]:
hour = [x for x in range(0,24)]
temp = [x + np.random.normal(17,3)  for x in range(0,24)]
rh = [x + np.random.normal(90,5)  for x in range(0,24)]
dew = [x + np.random.normal(-3,1)  for x in range(0,24)]

mock_data = pd.DataFrame({'hour': hour,
                        'inside_temp_hr': temp,
                         'inside_rh_hr': rh,
                         'temp_dew_diff': dew}
                        )

In [ ]:
mock_data
   

In [ ]:
risk_score, ev_hours = model_risk_scores(mock_data, 'botrytis', disease_dict=disease_temp_dict)

In [ ]:
risk_score

In [ ]:
x = [0.0, 2.0, 3.0, 2.0, 3.0, 2.0, 4.0, 3.0, 5.0, 3.0, 5.0, 5.0, 6.0, 5.0, 6.0, 9.0, 7.0, 9.0, 7.0, 7.0, 8.0, 8.0, 10.0, 12.0]

In [ ]:
sum(x)

In [ ]:
sum([2.0, 6.0, 6.0, 6.0, 5.0, 5.0, 5.0])

In [ ]:
y =[0.0, 0, 0, 2.0, 0, 0.0, 2.0, 0, 4.0, 6.0, 0, 1.0, 6.0, 3.0, 3.0, 5.0, 4.0, 5.0, 5.0, 8.0, 9.0, 9.0, 8.0, 11.0]
x=[0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
for x_, y_ in zip(x,y):
    print(x_, y_)
    #print(x_+y_)